In [1]:
import time 
import multiprocessing as mp
from multiprocessing import Pool
from collections import defaultdict
import sys
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts')
sys.path.append('/home/ak/Documents/PaperCode/stylised_facts/lob_for_futures')

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import lob_for_futures as lobFut
from lob_for_futures import *

from scipy.stats import jarque_bera
import matplotlib.pyplot as plt

from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch
import pickle
import datetime as dt
from datetime import timedelta
import glob
import fathon
from fathon import fathonUtils as fu
import itertools
import pyinform as pyinf
# import pingouin as pig
import seaborn as sns
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
from scipy.special import kl_div
import time
import pickle as pkl
import matplotlib.ticker as mtick

from functools import partial

def returns(s):

    # compute log returns
    arr = np.diff(np.log(s))

    return (pd.Series(arr, index=s.index[1:]))

ModuleNotFoundError: No module named 'lob_for_futures'

In [ ]:
# Lets create a function that shifted your dataframe first before calling the corr(). 
# Let us break down what we hope to accomplish, and then translate that into code. 
# For each hour (row) and for a given feature (column) I would like to find the value 
# for that feature N hours prior. 
# For each value of N (1-6 in our case) I want to make a new column 
# for that feature representing the Nth prior hour's measurement.

def df_derived_by_shift(df,lag=0,NON_DER=[]):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in NON_DER:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pd.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=i)
            i+=1
        df = pd.concat([df, dfn], axis=1)
    return df

def get_test_stats(bar_types,bar_returns,test_func,*args,**kwds):
    
    dct = {bar:(int(bar_ret.shape[0]), test_func(bar_ret,*args,**kwds)) 
           for bar,bar_ret in zip(bar_types,bar_returns)}
    df = (pd.DataFrame.from_dict(dct)
          .rename(index={0:'sample_size',1:f'{test_func.__name__}_stat'})
          .T)
    return df

In [ ]:
loc_activityClockData = '/media/ak/My Passport/ActivityClockData'
loc_ProcessedDataForExperiments = '/media/ak/My Passport1/ExperimentData/ProcessedDataForExperiments'
list_activityClockData = os.listdir('/media/ak/My Passport1/ActivityClockData')

os_data = '/media/ak/OS/Data'
FuturesDataSemiProcessed = os.path.join(os_data, 'FuturesDataSemiProcessed')
bars = ['tick', 'volume', 'calendar', 'dollar']

In [ ]:
symbols = os.listdir(FuturesDataSemiProcessed)
jb1_data_loc = os.path.join(FuturesDataSemiProcessed, 'JB1')
symbol_data_list = sorted(os.listdir(jb1_data_loc))
symbol_data_dates = [f.split('.')[0] for f in symbol_data_list if str('.pkl') in f]

In [ ]:
file_idx = 0
file_loc = os.path.join(jb1_data_loc, symbol_data_list[file_idx])
pickled_default_dict = pd.read_pickle(file_loc)
date = list(pickled_default_dict.keys())


In [ ]:
loaded_pickle_file = pd.read_pickle(file_loc)

In [ ]:
keys = list(loaded_pickle_file.keys())
print(keys)

In [ ]:
all_dfs = loaded_pickle_file[keys[0]]
all_dfs.keys()

In [ ]:
tick_df = all_dfs['tick']
volume_df = all_dfs['volume']
calendar_df = all_dfs['calendar']
dollar_df = all_dfs['dollar']

In [ ]:
dollar_df.columns.values # do cross correlation here

In [ ]:
volume_df_micro_structure = lobFut.apply_micro_structure_features(volume_df)

In [ ]:
volume_df_micro_structure

In [ ]:
volume_df_vol_estimation = lobFut.volatilityEstimation(volume_df)
tick_df_vol_estimation = lobFut.volatilityEstimation(tick_df)

In [ ]:
plt.hist(tick_df_vol_estimation.garmanKlass(5))

In [ ]:
import seaborn as sns

In [ ]:
sns.histplot(tick_df_vol_estimation.garmanKlass(5))